In [1]:
from IPython import get_ipython
from IPython.display import display
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
import numpy as np
import threading
import os
import csv
import datetime
from concurrent.futures import ThreadPoolExecutor
from IPython import get_ipython
from IPython.display import display
import spacy
nlp = spacy.load('en_core_web_sm')

df = pd.DataFrame(columns=['url', 'title', 'job', 'company', 'location', 'level', 'applicants', 'date', 'body', 'jobpage_html'])
df = df.fillna('null').replace([None], ['null'])
tmpz = 1
# functions

def find_between(s, first, last):
    while True:
        try:
            start = s.index(first) + len(first)
            end = s.index(last, start)
            return s[start:end]
        except ValueError:
            break

def get_jobs(urls):
    global df
    global start_str_jobs
    global end_str_jobs
    max_retries=10
    scraped_urls = set()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6446.71 Safari/537.36'
    }
    for url in urls:
        retry_count = 0
        while retry_count < max_retries:
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status()
                jobs_html = response.text

                # saves jobs page as txt
                if url == urls[0]:
                  with open('jobs_html.txt', 'w', encoding='utf-8') as f:
                      f.write(jobs_html)


                start_matches = re.findall(start_str_jobs, jobs_html, re.IGNORECASE)
                end_matches = re.findall(end_str_jobs, jobs_html, re.IGNORECASE)
                for start_match in start_matches:
                      extracted_url = find_between(jobs_html, start_str_jobs, end_str_jobs)
                      if extracted_url:
                          extracted_url = (start_str_jobs + extracted_url)[6:]
                          scraped_urls.add(extracted_url)
                          jobs_html = jobs_html.replace(extracted_url, "", 1)
                # saves urls in csv
                break
            except requests.exceptions.RequestException as e:
                retry_count += 1
                time.sleep(2)
            else:
                print(f"Maximum retries reached for URL {url}. Skipping...")
    scraped_urls = list(set(scraped_urls))
    for extracted_url in scraped_urls:
        df.loc[len(df), ['url']] = [extracted_url]

def get_job_posting(max_retries=8, backoff_factor=2):
    global df
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.6446.71 Safari/537.36'
    }

    def process_url(url, index):
        retry_count = 0
        while retry_count < max_retries:
            try:
                response = requests.get(url, headers=headers)
                response.raise_for_status()
                df.loc[index, 'jobpage_html'] = response.text
                get_title(index)
                get_body(index)
                get_job_level(index)
                get_job_date(index)
                get_job_applicants(index)
                break
            except requests.exceptions.RequestException as e:
                retry_count += 1
                if isinstance(e, requests.exceptions.HTTPError) and e.response.status_code == 429:
                    wait_time = backoff_factor * retry_count
                    print(f"Rate limit exceeded. Retrying in {wait_time} seconds.")
                    time.sleep(wait_time)
                else:
                    print(f"Error fetching URL {url}: {e}")
                    break

    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(process_url, row['url'], index) for index, row in df.iterrows()]
        for future in futures:
            future.result()

def get_title(index):
    global df
    global start_str_title
    global end_str_title
    global values_to_be_removed
    try:
        if isinstance(df.loc[index, "jobpage_html"], str):
            df.loc[index, "title"] = find_between(df.loc[index, "jobpage_html"], start_str_title, end_str_title)
        else:
            df.loc[index, "title"] = "Value Error occurred"  # Or handle it as you prefer
    except ValueError:
        print("Value Error occurred. Could not find the desired substring. Failed at extracting the body")
        df.loc[index, "title"] = "Value Error occurred"
    df.loc[index, "title"] = re.sub("|".join(values_to_be_removed), "", df.loc[index, "title"])
    try:
        first_split = df.loc[index, "title"].split(" hiring ", 1)
        df.loc[index, "company"] = first_split[0]
        second_split = first_split[1].split(" in ", 1)
        df.loc[index, "job"] = second_split[0]
        df.loc[index, "location"] = second_split[1][:-11]
    except (ValueError, IndexError) as e:
        print(f"Error splitting string: {e}")
        df.loc[index, "title"] = ['','']

def get_body(index):
    global df
    global start_str_body
    global end_str_body
    global values_to_be_removed
    try:
        df.loc[index, "body"] = find_between(df.loc[index, "jobpage_html"], start_str_body, end_str_body)
        df.loc[index, "body"] = df.loc[index, "body"][10:-55]
    except ValueError:
        print("Value Error occurred. Could not find the desired substring. Failed at extracting the body")
        df.loc[index, "body"] = "Value Error occurred"
    df.loc[index, "body"] = re.sub("|".join(values_to_be_removed), "", df.loc[index, "body"])


def get_job_applicants(index):
    global df
    global start_str_applicants
    global end_str_applicants
    global values_to_be_removed
    try:
        df.loc[index, "applicants"] = find_between(df.loc[index, "jobpage_html"], start_str_applicants, end_str_applicants)
        df.loc[index, "applicants"] = df.loc[index, "applicants"]
    except IndexError:
        df.loc[index, "applicants"] = "Value Error occurred"

def get_job_date(index):
    global df
    global start_str_date
    global end_str_date
    try:
        df.loc[index, "date"] = find_between(df.loc[index, "jobpage_html"], start_str_date, end_str_date)
        if df.loc[index, "date"] is not None:
            df.loc[index, "date"] = df.loc[index, "date"].split(" ago")[0] + ' ago'
        else:
            df.loc[index, "date"] = "Value Error occurred" # or any other default value you want
    except ValueError:
        df.loc[index, "date"]  = "Value Error occurred"

def get_job_level(index):
    global df
    global start_str_level
    global end_str_level
    global values_to_be_removed
    try:
        df.loc[index, "level"] = find_between(df.loc[index, "jobpage_html"], start_str_level, end_str_level)
    except ValueError:
        df.loc[index, "level"]  = "Value Error occurred"
    df.loc[index, "level"]  = re.sub("|".join(values_to_be_removed), "", df.loc[index, "level"] )
    df.loc[index, "level"] = df.loc[index, "level"].strip()

def get_min_years(df):
  df['min_years'] = None
  for x in range(len(df['body'])):
    try:
      doc = nlp(df['body'][x])
    except Exception as e:
      continue
    for ent in doc.ents:
      if ent.label_ == "DATE" and 'year' in ent.text.lower():
        for token in ent:
          if token.pos_ == 'NUM':
            try:
              token_float = float(token.text)
            except ValueError:
              continue
            if pd.isna(df['min_years'][x]) or token_float < df['min_years'][x]:
              df['min_years'][x] = token_float

def get_min_years(df):
  df['min_years'] = None
  for x in range(len(df['body'])):
    try:
      # Ensure df['body'][x] is a string before processing
      if isinstance(df['body'][x], str):
        doc = nlp(df['body'][x])
      else:
        continue  # Skip if not a string
    except Exception as e:
      continue
    for ent in doc.ents:
      if ent.label_ == "DATE" and 'year' in ent.text.lower():
        for token in ent:
          if token.pos_ == 'NUM':
            try:
              token_float = float(token.text)
            except ValueError:
              continue
            if pd.isna(df['min_years'][x]) or token_float < df['min_years'][x]:
              df['min_years'][x] = token_float
  return df #Ensure the df is returned

def score_date(df):
    for x in range(len(df)):
        try:
            if 'minute' in df['date'][x].lower():
                df.loc[x, 'score'] += 12.0
            elif 'hour' in df['date'][x].lower():
                df.loc[x, 'score'] += 10.0
            elif 'yesterday' in df['date'][x].lower():
                df.loc[x, 'score'] += 8.0
            elif '1 day' in df['date'][x].lower():
                df.loc[x, 'score'] += 8.0
            elif '2 day' in df['date'][x].lower():
                df.loc[x, 'score'] += 2.0
            elif '3 day' in df['date'][x].lower():
                df.loc[x, 'score'] += 0
            elif '4 day' in df['date'][x].lower():
                df.loc[x, 'score'] -= 2.0
            elif '5 day' in df['date'][x].lower():
                df.loc[x, 'score'] -= 5.0
            elif '6 day' in df['date'][x].lower():
                df.loc[x, 'score'] -= 7.0
            elif 'wk ago' in df['date'][x].lower():
                df.loc[x, 'score'] -= 10.0
            elif 'mo ago' in df['date'][x].lower():
                df.loc[x, 'score'] -= 115.0
        except Exception:
            pass

def score_keyword(df):
    for x in range(len(df)):
        try:
          if 'data' in df['job'][x].lower() and 'analy' in df['job'][x].lower():
            df['score'][x] += 10.0
          elif 'data' in df['job'][x].lower() and 'scien' in df['job'][x].lower():
            df['score'][x] += 10.0
          elif ('data' in (df['job'][x].lower())):
            df['score'][x] += 5.0
          elif ('analyst' in (df['job'][x].lower())):
            df['score'][x] += 2.0
          if ('excel' in (df['job'][x].lower())):
            df['score'][x] += 2.0
          if ('python' in (df['job'][x].lower())):
            df['score'][x] += 2.0
          if ('data entry' in df['job'][x].lower() and 'entry level' in df['job'][x].lower()):
            df['score'][x] -= 12.0
          elif ('data entry' in (df['job'][x].lower())):
            df['score'][x] -= 22.0
          if ('entry level' in (df['job'][x].lower())):
            df['score'][x] += 25.0
          elif ('entry' in (df['job'][x].lower())):
            df['score'][x] += 20.0
          if ('principal' in (df['job'][x].lower())):
            df['score'][x] -= 200.0
          if ('senior' in (df['job'][x].lower())):
            df['score'][x] -= 200.0
          if ('sr.' in (df['job'][x].lower())):
            df['score'][x] -= 200.0
          if ('lead' in (df['job'][x].lower())):
            df['score'][x] -= 200.0
          else:
            if df['level'][x] == 'Mid-Senior level':
              df['score'][x] = df['score'][x] - 200.0
            if df['level'][x] == 'Associate':
              df['score'][x] = df['score'][x] - 5.0
        except Exception:
            pass

def score_pay(df):
    for x in range(len(df)):
      try:
        tmp = df['pay'][x].lower()
      except:
        try:
          tmp = str(df['pay'][x])
        except:
          return None
      try:
        if ('hr' in (tmp)):
          numbers = re.findall(r'\d+\.?\d*', tmp)
          float_numbers = [float(num.replace(',', '')) for num in numbers]
          lowest_number = min(float_numbers)
          if lowest_number >= 50:
            df['score'][x] -= 15.0
        elif ('k' in (tmp)):
          numbers = re.findall(r'\d+\.?\d*', tmp)
          float_numbers = [float(num.replace(',', '')) for num in numbers]
          lowest_number = min(float_numbers)
          if lowest_number >= 100:
            df['score'][x] -= 15.0
        elif ('yr' in (tmp) or 'year' in (tmp)):
          numbers = re.findall(r'\d+\.?\d*', tmp)
          float_numbers = [float(num.replace(',', '')) for num in numbers]
          lowest_number = min(float_numbers)
          if lowest_number >= 1000:
            if lowest_number >= 100:
              df['score'][x] -= 15.0
          elif lowest_number >= 100000:
            df['score'][x] -= 15.0
      except:
          pass
# ----------------------------------------------------------------------------------------------------------------------------------------------------


#variables
start_str_jobs = 'href="https://www.linkedin.com/jobs/view/'
end_str_jobs = '"'
start_str_body = 'relative overflow-hidden">'
end_str_body = '<button class="show-more-less-html__button show-more-less-button'
start_str_title = '<title>'
end_str_title = '</title>'
start_str_applicants = '<figcaption class="num-applicants__caption">v'
end_str_applicants = '</figcaption>'
start_str_date = '<span class="posted-time-ago__text topcard__flavor--metadata">'
end_str_date = '</time>'
start_str_level = '<span class="description__job-criteria-text description__job-criteria-text--criteria">'
end_str_level = '</span>'
values_to_be_removed = ['</em>', '<em>', '<strong>', '</strong>', '<p>', '</p>', '<br>', '</br>', '<li>', '</li>', '<u>', '</u>', '<ul>', '</ul>']
failed_url = 0
success_url = 0
threads = []
num_threads = 10

# data analyst pleasantville
url1 = 'https://www.linkedin.com/jobs/search/?currentJobId=4035449037&distance=25&f_E=1%2C2%2C3&f_TPR=r604800&geoId=101271451&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true'
url2 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036976162&distance=25&f_E=1%2C2%2C3&f_TPR=r604800&geoId=101271451&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&start=25'
url3 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036976162&distance=25&f_E=1%2C2%2C3&f_TPR=r604800&geoId=101271451&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&start=50'
url4 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036976162&distance=25&f_E=1%2C2%2C3&f_TPR=r604800&geoId=101271451&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&start=75'
# data science pleasantville
url5 = 'https://www.linkedin.com/jobs/search/?currentJobId=4014188659&distance=25&f_E=1%2C2%2C3&f_TPR=r604800&geoId=101271451&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true'
url6 = 'https://www.linkedin.com/jobs/search/?currentJobId=4002447259&distance=25&f_E=1%2C2%2C3&f_TPR=r604800&geoId=101271451&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=25'
url7 = 'https://www.linkedin.com/jobs/search/?currentJobId=4002447259&distance=25&f_E=1%2C2%2C3&f_TPR=r604800&geoId=101271451&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=50'
url8 = 'https://www.linkedin.com/jobs/search/?currentJobId=4002447259&distance=25&f_E=1%2C2%2C3&f_TPR=r604800&geoId=101271451&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=75'
# data analyst NYC
url9 = 'https://www.linkedin.com/jobs/search/?currentJobId=4038218781&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true'
url10 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036976162&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=25'
url11 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036976162&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=50'
url12 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036976162&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=75'
url13 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036976162&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=100'
url14 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036976162&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=125'
url15 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036976162&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=150'
url16 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036976162&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_LOCATION_AUTOCOMPLETE&refresh=true&start=175'


# data science NYC
url17 = 'https://www.linkedin.com/jobs/search/?currentJobId=4038218781&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20science&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true'
url18 = 'https://www.linkedin.com/jobs/search/?currentJobId=4038542149&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20science&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true&start=25'
url19 = 'https://www.linkedin.com/jobs/search/?currentJobId=4038542149&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20science&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true&start=50'
url20 = 'https://www.linkedin.com/jobs/search/?currentJobId=4038542149&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20science&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true&start=75'
url21 = 'https://www.linkedin.com/jobs/search/?currentJobId=4038542149&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20science&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true&start=100'
url22 = 'https://www.linkedin.com/jobs/search/?currentJobId=4038542149&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20science&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true&start=125'
url23 = 'https://www.linkedin.com/jobs/search/?currentJobId=4038542149&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20science&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true&start=150'
url24 = 'https://www.linkedin.com/jobs/search/?currentJobId=4038542149&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20science&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true&start=175'
url25 = 'https://www.linkedin.com/jobs/search/?currentJobId=4038542149&f_E=1%2C2%2C3&f_TPR=r604800&geoId=102571732&keywords=data%20science&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true&start=200'

# data scientist stamford
url26 = 'https://www.linkedin.com/jobs/search/?currentJobId=4040730828&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20science&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true'
url27 = 'https://www.linkedin.com/jobs/search/?currentJobId=4040732977&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20science&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=25'
url28 = 'https://www.linkedin.com/jobs/search/?currentJobId=4040732977&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20science&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=50'
url29 = 'https://www.linkedin.com/jobs/search/?currentJobId=4040732977&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20science&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=75'
url30 = 'https://www.linkedin.com/jobs/search/?currentJobId=4040732977&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20science&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=100'
url31 = 'https://www.linkedin.com/jobs/search/?currentJobId=4040732977&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20science&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=125'
url32 = 'https://www.linkedin.com/jobs/search/?currentJobId=4040732977&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20science&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=150'
url33 = 'https://www.linkedin.com/jobs/search/?currentJobId=4040732977&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20science&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=175'
url34 = 'https://www.linkedin.com/jobs/search/?currentJobId=4040732977&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20science&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=200'

# data analyst stamford
url36 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036991331&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true'
url36 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036991331&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=25'
url37 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036991331&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=50'
url38 = 'https://www.linkedin.com/jobs/search/?currentJobId=4036991331&f_E=1%2C2%2C3&f_TPR=r604800&geoId=106935132&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true&start=75'

# ----------------------------------------------------------------------------------------------------------------------------------------------------


# program
list_jobs_urls = [globals()[var] for var in globals() if var.startswith('url')]
max_retries = 5  # Maximum number of retries
for retry_count in range(max_retries):
    df = pd.DataFrame(columns=['score', 'url', 'title', 'job', 'company', 'location', 'level', 'applicants', 'date', 'body', 'jobpage_html'])  # Reset the DataFrame for each retry
    get_jobs(list_jobs_urls)
    df = df.drop_duplicates(subset=['url'])
    max_jobs = str(len(df))
    print('---- Jobs retrieved. urls: ' + max_jobs)

    if int(max_jobs) >= 40:
        print("Sufficient jobs retrieved. Proceeding...")
        break  # Exit the retry loop if enough jobs are found
    else:
        print(f"Insufficient jobs retrieved ({max_jobs}). Retrying... (Attempt {retry_count + 1}/{max_retries})")
get_job_posting()
df = df.drop('jobpage_html', axis=1)
# ----------------------------------------------------------------------------------------------------------------------------------------------------
def score(df):
  df = score_name(df)
  #df = score_level(df) # Keep commented as it was in the original code

  # Iterate through each row of the DataFrame
  for index, row in df.iterrows():
    # Check if 'title', 'job', and 'company' are all null for the current row
    if pd.isna(row['title']) and pd.isna(row['job']) and pd.isna(row['company']):
      # Assign a score of -100 to the 'score' column for that row
      df.loc[index, 'score'] = -100

  return df

def score_name(df):
  # If 'score' column doesn't exist, create it and initialize to 0
  if 'score' not in df.columns:
    df['score'] = 0
  for index in df.index:  # Iterate using df.index to ensure correct access
    # Check if 'url' is not null before accessing it
    if pd.notna(df.loc[index, 'url']):
        url_value = df.loc[index, 'url'].lower()  # Access using .loc for safety
        if 'data' in url_value and 'analyst' in url_value:  # Use separate checks
          df.loc[index, 'score'] += 10.0
        elif 'analy' in url_value:
          df.loc[index, 'score'] += 5
        elif 'data' in url_value:
          df.loc[index, 'score'] += 7
        if 'entry' in url_value:
          df.loc[index, 'score'] += 10.0
        if 'data-entry' in url_value:
          df.loc[index, 'score'] -= 15.0
        if 'senior' in url_value:
          df.loc[index, 'score'] -= 10.0
        if 'principal' in url_value:
          df.loc[index, 'score'] -= 10.0
        if 'data' in url_value and 'scien' in url_value:  # Use separate checks
          df.loc[index, 'score'] += 10.0
  return df

def score_level(df):
  # Check if the 'level' column exists and is not empty before converting to string
  if 'level' in df.columns and not df['level'].empty:
      df['level'] = df['level'].astype(str) # Convert to string using astype
      for index in df.index:
        # Check if the value in 'level' is not null before lowercasing
        if pd.notna(df.loc[index, 'level']):
            if 'entry level' == df.loc[index, 'level'].lower():
              df.loc[index, 'score'] += 10.0
            elif 'associate' == df.loc[index, 'level'].lower():
              df.loc[index, 'score'] += 10.0
            elif 'internship' == df.loc[index, 'level'].lower():
              df.loc[index, 'score'] -= 10.0
            elif 'senior' == df.loc[index, 'level'].lower():
              df.loc[index, 'score'] -= 50.0
  return df # Ensure the df is returned
df['score'] = 0
df = df.fillna(np.nan)  # Replace all NaN values with np.nan
df = df.replace([None], np.nan)  # Replace all None values with np.nan
df = df.replace(['Value Error occurred'], np.nan)  # Replace all  'Value Error occurred' values with np.nan
df = score(df)

df = df.sort_values(by='score', ascending=False)
df.to_csv('linkedin_jobs.csv', index=False)


Streaming output truncated to the last 5000 lines.
Rate limit exceeded. Retrying in 14 seconds.
Rate limit exceeded. Retrying in 4 seconds.
Rate limit exceeded. Retrying in 4 seconds.
Rate limit exceeded. Retrying in 4 seconds.
Rate limit exceeded. Retrying in 4 seconds.
Rate limit exceeded. Retrying in 2 seconds.
Rate limit exceeded. Retrying in 4 seconds.
Rate limit exceeded. Retrying in 4 seconds.
Rate limit exceeded. Retrying in 2 seconds.
Rate limit exceeded. Retrying in 4 seconds.
Rate limit exceeded. Retrying in 4 seconds.
Rate limit exceeded. Retrying in 12 seconds.
Rate limit exceeded. Retrying in 4 seconds.
Rate limit exceeded. Retrying in 12 seconds.
Rate limit exceeded. Retrying in 14 seconds.
Rate limit exceeded. Retrying in 14 seconds.
Rate limit exceeded. Retrying in 6 seconds.
Rate limit exceeded. Retrying in 6 seconds.
Rate limit exceeded. Retrying in 2 seconds.
Rate limit exceeded. Retrying in 6 seconds.
Rate limit exceeded. Retrying in 2 seconds.
Rate limit exceeded.

/tmp/ipython-input-487154966.py:472: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(np.nan)  # Replace all NaN values with np.nan


In [ ]:
# Select the first row of the DataFrame and get the 'jobpage_html' value
single_job_html = df.loc[0, 'jobpage_html']

# Define the filename
filename = 'single_job_posting.txt'

# Write the HTML content to the file
with open(filename, 'w', encoding='utf-8') as f:
    f.write(single_job_html)

print(f"Saved single job posting to {filename}")

Saved single job posting to single_job_posting.txt
